5.	Visualize a decision tree that predicts whether tennis will be played on a particular day. Find out the root node of that tree. Use (play_tennis.xslx) dataset for creating the classification tree. Solve this problem manually using the ID3 algorithm.

In [10]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import math

filepath = "./play_tennis.xlsx"
data = pd.read_excel(filepath)
data.head()

,Outlook,Temperature,Humidity,Windy,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rainy,Mild,High,Weak,Yes
4,Rainy,Cool,Normal,Weak,Yes


In [17]:


def decision_tree(df, val = None):

    def feature_entropy(df):
        ans = {}
        for feature in df.keys()[:-1]:
            l = len(df)
            y_uniques = df.iloc[:, -1].unique()
            x_uniques = df[feature].unique()
            y_str = df.keys()[-1]
            val = 0
            total=[]
            e = 10.e-10
            for x in x_uniques:
                d = df[feature][df[feature] == x].count() + e
                for y in y_uniques:
                    n = df[feature][df[feature] == x][df[y_str] == y].count()
                    val+= abs(n/d*math.log2((n/d) + e)*d/l)
            val2 = 0
            for i in df[y_str].value_counts():
                val2+= abs(i/(l)*math.log2((i/(l))))
            
            ans[feature] = val2 - val

        return (dict(sorted(ans.items(), key=lambda x: x[1], reverse=True)))
    
    entropy = feature_entropy(df)
    entropy_keys = entropy.keys()
    try:
        node = list(entropy_keys)[0]
    except:
        return {}
    node_cat = df[node].unique()
    y_str = df.keys()[-1]

    if val is None:
        val = {}
        val[node] = {}
        
    for i in node_cat:
        data = df[df[node] == i].reset_index(drop =True).drop(node, axis = 1)
        if len(data[y_str]) == 1:
            val[node][i] = data[y_str][0]
        else:
            val[node][i] = decision_tree(data)

    return val

def classifier(test,tree_dict):
    feature = next(iter(tree_dict)) 
    if test[feature] in tree_dict[feature].keys():

        result = tree_dict[feature][test[feature]]
        if isinstance(result, dict):
            return classifier(test, result)
        else:
            return result

input={'Outlook':'Rainy','Temperature':'Mild','Humidity':'High','Windy':'Weak'}
print("--------------------------------------------------------------")
print("Decision Tree Classifier")
print("For the Following input: ", input)
print("Result is: ",classifier(input, decision_tree(data)))
print("--------------------------------------------------------------")
print("Tree Decision Visualization:",(decision_tree(data)))
print("--------------------------------------------------------------")


    

    


--------------------------------------------------------------
Decision Tree Classifier
For the Following input:  {'Outlook': 'Rainy', 'Temperature': 'Mild', 'Humidity': 'High', 'Windy': 'Weak'}
Result is:  Yes
--------------------------------------------------------------
Tree Decision Visualization: {'Outlook': {'Sunny': {'Humidity': {'High': {'Temperature': {'Hot': {'Windy': {'Weak': 'No', 'Strong': 'No'}}, 'Mild': 'No'}}, 'Normal': {'Temperature': {'Cool': 'Yes', 'Mild': 'Yes'}}}}, 'Overcast': {'Temperature': {'Hot': {'Humidity': {'High': 'Yes', 'Normal': 'Yes'}}, 'Cool': 'Yes', 'Mild': 'Yes'}}, 'Rainy': {'Windy': {'Weak': {'Temperature': {'Mild': {'Humidity': {'High': 'Yes', 'Normal': 'Yes'}}, 'Cool': 'Yes'}}, 'Strong': {'Temperature': {'Cool': 'No', 'Mild': 'No'}}}}}}
--------------------------------------------------------------
